### Roman Imaging WCS

#### Generalized World Coordinate System (GWCS) Overview

Roman and JWST use the [GWCS](https://gwcs.readthedocs.io/en/latest/) for managing the World Coordinate System WCS). We call "WCS" the mapping from "pixel" coordinates to some "real-world" physical coordinates - celestial, spectral, time, etc. GWCS is a generalized implementation of WCS aiming to mitigate the limitations of the FITS WCS standard. It is a flexible toolkit for expressing and evaluating transformations between pixel and world coordinates, as well as intermediate coordinates. The GWCS object supports a data model which includes the entire transformation pipeline from input pixel coordinates to world coordinates (and vice versa). 

GWCS is based on astropy and supports the [Shared Interface for WCS](https://github.com/astropy/astropy-APEs/blob/main/APE14.rst). The WCS "pipeline" is a list of steps, where each step is a tuple of coordinate frame and a transform to the next frame. 

Transforms are based on [astropy.modeling](https://docs.astropy.org/en/stable/modeling/) and include [unit](https://docs.astropy.org/en/stable/units/index.html) support. Coordinate frames utilize [astropy.coordinates](https://docs.astropy.org/en/stable/coordinates/index.html). 
The GWCS object is serialized to [ASDF](https://asdf-standard.readthedocs.io/en/latest/) using the ASDF [WCS](https://asdf-wcs-schemas.readthedocs.io/en/latest/) and [transforms](https://asdf-transform-schemas.readthedocs.io/en/latest/) extensions.

#### Roman Imaging WCS

The `assign_wcs` step in the Roman calibration pipeline constructs a WCS object and assigns it to the Level 2 image. The distortion transformations are stored in a reference file in CRDS, with `reftype=distortion`. The distortion includes all transformsations from a detector to a coordinate frame associated with the telescope `(V2, V3)`. The telescope telemetry is used to transform from `(V2, V3)` to celestial coordinates.



We can use `asdf` or `roman_datamodels` to open the file and retrieve the WCS object.

In [1]:
from roman_datamodels import datamodels as rdm

In [2]:
image = rdm.open('r002_assign_wcs.asdf')

image.search('wcs')

root (AsdfObject)
└─roman (WfiImage) # The schema for WFI Level 2 images.

  └─meta (dict)
    ├─cal_step (CalStep) # Calibration Status
    │ └─assign_wcs (str): COMPLETE # Assign World Coordinate System
    ├─wcs (WCS)
    └─wcsinfo (Wcsinfo) # WCS parameters

In [3]:
w = image.meta.wcs

`print` shows the WCS pipeline, while `repr` prints all transforms.

In [5]:
print(w)

  From     Transform  
-------- -------------
detector CompoundModel
    v2v3      v23tosky
   world          None


In [6]:
w

<WCS(output_frame=world, input_frame=detector, forward_transform=Model: CompoundModel
Inputs: ('x0', 'x1')
Outputs: ('lon', 'lat')
Model set size: 1
Expression: [0] & [1] | [2] & [3] | [4] | [5] & [6] | [7] | [8] & [9] | [10] & [11] | [12] & [13] | [14] | [15] | [16]
Components: 
    [0]: <Shift(offset=1.)>

    [1]: <Shift(offset=1.)>

    [2]: <Shift(offset=-2044.5)>

    [3]: <Shift(offset=-2044.5)>

    [4]: <Mapping((0, 1, 0, 1))>

    [5]: <Polynomial2D(5, c0_0=0., c1_0=0.10900268, c2_0=-0.00000016, c3_0=-0., c4_0=0., c5_0=0., c0_1=0.00128425, c0_2=-0.00000005, c0_3=-0., c0_4=0., c0_5=0., c1_1=0.00000011, c1_2=-0., c1_3=0., c1_4=-0., c2_1=-0., c2_2=-0., c2_3=0., c3_1=-0., c3_2=-0., c4_1=0.)>

    [6]: <Polynomial2D(5, c0_0=0., c1_0=0.00125276, c2_0=0.00000005, c3_0=-0., c4_0=0., c5_0=0., c0_1=0.10887167, c0_2=0.00000017, c0_3=-0., c0_4=0., c0_5=-0., c1_1=-0.0000001, c1_2=-0., c1_3=0., c1_4=-0., c2_1=-0., c2_2=-0., c2_3=-0., c3_1=0., c3_2=0., c4_1=-0.)>

    [7]: <Mapping((0, 1, 0

The WCS can be evaluated directly as a function or using the Shared WCS Interface methods

In [7]:
from gwcs.wcstools import grid_from_bounding_box

The `bounding_box` of a WCS object represents the range of input values over which the transforms are valid. Typically it is the full detector. By default, input values outside the `bounding box` return `NaN`s. 

In [9]:
w.bounding_box

ModelBoundingBox(
    intervals={
        x0: Interval(lower=-0.5, upper=4087.5)
        x1: Interval(lower=-0.5, upper=4087.5)
    }
    model=CompoundModel(inputs=('x0', 'x1'))
    order='C'
)

In [10]:
x, y = grid_from_bounding_box(w.bounding_box)

Evaluating the WCS object returns the numerical values of the result.

In [11]:
ra, dec = w(x, y)
print (ra, dec)

[[84.12146342 84.12141994 84.12137645 ... 83.94316241 83.94311863
  83.94307486]
 [84.12138907 84.12134559 84.1213021  ... 83.94308787 83.94304409
  83.94300032]
 [84.12131473 84.12127124 84.12122775 ... 83.94301333 83.94296955
  83.94292578]
 ...
 [83.81675817 83.81671447 83.81667077 ... 83.63789396 83.6378501
  83.63780624]
 [83.81668342 83.81663971 83.81659601 ... 83.63781911 83.63777525
  83.63773139]
 [83.81660866 83.81656496 83.81652126 ... 83.63774425 83.63770039
  83.63765653]] [[-69.30432586 -69.30435206 -69.30437827 ... -69.41052071 -69.41054647
  -69.41057224]
 [-69.30431122 -69.30433742 -69.30436362 ... -69.41050617 -69.41053193
  -69.4105577 ]
 [-69.30429657 -69.30432277 -69.30434897 ... -69.41049163 -69.4105174
  -69.41054316]
 ...
 [-69.24365518 -69.24368144 -69.24370771 ... -69.350207   -69.35023287
  -69.35025874]
 [-69.24364013 -69.2436664  -69.24369266 ... -69.35019202 -69.35021789
  -69.35024376]
 [-69.24362508 -69.24365135 -69.24367762 ... -69.35017704 -69.35020291

Using the Shared WCS interface methods returns a Python object of type astropy.coordinates.SkyCoord.

In [12]:
sky = w.pixel_to_world(x, y)

In [13]:
print(sky)

<SkyCoord (ICRS): (ra, dec) in deg
    [[(84.12146342, -69.30432586), (84.12141994, -69.30435206),
      (84.12137645, -69.30437827), ..., (83.94316241, -69.41052071),
      (83.94311863, -69.41054647), (83.94307486, -69.41057224)],
     [(84.12138907, -69.30431122), (84.12134559, -69.30433742),
      (84.1213021 , -69.30436362), ..., (83.94308787, -69.41050617),
      (83.94304409, -69.41053193), (83.94300032, -69.4105577 )],
     [(84.12131473, -69.30429657), (84.12127124, -69.30432277),
      (84.12122775, -69.30434897), ..., (83.94301333, -69.41049163),
      (83.94296955, -69.4105174 ), (83.94292578, -69.41054316)],
     ...,
     [(83.81675817, -69.24365518), (83.81671447, -69.24368144),
      (83.81667077, -69.24370771), ..., (83.63789396, -69.350207  ),
      (83.6378501 , -69.35023287), (83.63780624, -69.35025874)],
     [(83.81668342, -69.24364013), (83.81663971, -69.2436664 ),
      (83.81659601, -69.24369266), ..., (83.63781911, -69.35019202),
      (83.63777525, -69.350217

Other useful methods include

`getting a transform between intermediate frames`

In [14]:
detector_to_v2v3 = w.get_transform('detector', 'v2v3')
print(detector_to_v2v3)

Model: CompoundModel
Inputs: ('x0', 'x1')
Outputs: ('y0', 'y1')
Model set size: 1
Expression: [0] & [1] | [2] & [3] | [4] | [5] & [6] | [7] | [8] & [9] | [10] & [11]
Components: 
    [0]: <Shift(offset=1.)>

    [1]: <Shift(offset=1.)>

    [2]: <Shift(offset=-2044.5)>

    [3]: <Shift(offset=-2044.5)>

    [4]: <Mapping((0, 1, 0, 1))>

    [5]: <Polynomial2D(5, c0_0=0., c1_0=0.10900268, c2_0=-0.00000016, c3_0=-0., c4_0=0., c5_0=0., c0_1=0.00128425, c0_2=-0.00000005, c0_3=-0., c0_4=0., c0_5=0., c1_1=0.00000011, c1_2=-0., c1_3=0., c1_4=-0., c2_1=-0., c2_2=-0., c2_3=0., c3_1=-0., c3_2=-0., c4_1=0.)>

    [6]: <Polynomial2D(5, c0_0=0., c1_0=0.00125276, c2_0=0.00000005, c3_0=-0., c4_0=0., c5_0=0., c0_1=0.10887167, c0_2=0.00000017, c0_3=-0., c0_4=0., c0_5=-0., c1_1=-0.0000001, c1_2=-0., c1_3=0., c1_4=-0., c2_1=-0., c2_2=-0., c2_3=-0., c3_1=0., c3_2=0., c4_1=-0.)>

    [7]: <Mapping((0, 1, 0, 1))>

    [8]: <Polynomial2D(1, c0_0=0., c1_0=-0.5, c0_1=-0.8660254)>

    [9]: <Polynomial2D(1, c0_

`showing the coorindate frames in the WCS pipeline`

In [15]:
print(w.available_frames)

['detector', 'v2v3', 'world']


Retrieving the entire `forward` or `backward` transform

In [16]:
print(w.forward_transform)

Model: CompoundModel
Inputs: ('x0', 'x1')
Outputs: ('lon', 'lat')
Model set size: 1
Expression: [0] & [1] | [2] & [3] | [4] | [5] & [6] | [7] | [8] & [9] | [10] & [11] | [12] & [13] | [14] | [15] | [16]
Components: 
    [0]: <Shift(offset=1.)>

    [1]: <Shift(offset=1.)>

    [2]: <Shift(offset=-2044.5)>

    [3]: <Shift(offset=-2044.5)>

    [4]: <Mapping((0, 1, 0, 1))>

    [5]: <Polynomial2D(5, c0_0=0., c1_0=0.10900268, c2_0=-0.00000016, c3_0=-0., c4_0=0., c5_0=0., c0_1=0.00128425, c0_2=-0.00000005, c0_3=-0., c0_4=0., c0_5=0., c1_1=0.00000011, c1_2=-0., c1_3=0., c1_4=-0., c2_1=-0., c2_2=-0., c2_3=0., c3_1=-0., c3_2=-0., c4_1=0.)>

    [6]: <Polynomial2D(5, c0_0=0., c1_0=0.00125276, c2_0=0.00000005, c3_0=-0., c4_0=0., c5_0=0., c0_1=0.10887167, c0_2=0.00000017, c0_3=-0., c0_4=0., c0_5=-0., c1_1=-0.0000001, c1_2=-0., c1_3=0., c1_4=-0., c2_1=-0., c2_2=-0., c2_3=-0., c3_1=0., c3_2=0., c4_1=-0.)>

    [7]: <Mapping((0, 1, 0, 1))>

    [8]: <Polynomial2D(1, c0_0=0., c1_0=-0.5, c0_1=-0.866

In [17]:
print(w.backward_transform)

Model: CompoundModel
Inputs: ('lon', 'lat')
Outputs: ('y0', 'y1')
Model set size: 1
Expression: [0] | [1] | [2] | [3] & [4] | [5] & [6] | [7] | [8] & [9] | [10] | [11] & [12] | [13] & [14] | [15] & [16]
Components: 
    [0]: <SphericalToCartesian()>

    [1]: <RotationSequence3D(angles=[83.87999291, 69.32761623, -0.        , -0.47742993, -0.1488929 ])>

    [2]: <CartesianToSpherical()>

    [3]: <Scale(factor=3600.)>

    [4]: <Scale(factor=3600.)>

    [5]: <Shift(offset=-536.01443984)>

    [6]: <Shift(offset=1718.74775661)>

    [7]: <Mapping((0, 1, 0, 1))>

    [8]: <Polynomial2D(1, c0_0=0., c1_0=-0.5, c0_1=-0.8660254)>

    [9]: <Polynomial2D(1, c0_0=0., c1_0=-0.8660254, c0_1=0.5)>

    [10]: <Mapping((0, 1, 0, 1))>

    [11]: <Polynomial2D(5, c0_0=0., c1_0=9.17532754, c2_0=0.00012464, c3_0=0.00000004, c4_0=-0., c5_0=-0., c0_1=-0.10823031, c0_2=0.00004166, c0_3=0.00000001, c0_4=-0., c0_5=0., c1_1=-0.00008863, c1_2=0.00000002, c1_3=-0., c1_4=0., c2_1=0.00000002, c2_2=0., c2_3=-0.,

Other GWCS features include the ability to

- insert additional frames and transforms
- compute the inverse of the transforms using an iterative numerical method (in case an inverse transform is not provided)
- create  FITS approximation using the SIP convention to represent the distortion
- convert to other celestial frames usnig the `Shared API`
